In [27]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format
con = sqlite3.connect('../../datasets/checking-logs.sqlite')

In [28]:
df = pd.read_sql_query(
    '''
    SELECT 
        uid,
        timestamp
    FROM checker
    WHERE 
        uid LIKE 'user_%'
        AND status = 'ready'
        AND labname = 'project1'
    ''',
    con,
    parse_dates=['timestamp'],
)
df['date'] = df['timestamp'].dt.date
df

,uid,timestamp,date
0,user_4,2020-04-17 05:19:02.744528,2020-04-17
1,user_4,2020-04-17 05:22:45.549397,2020-04-17
2,user_4,2020-04-17 05:34:24.422370,2020-04-17
3,user_4,2020-04-17 05:43:27.773992,2020-04-17
4,user_4,2020-04-17 05:46:32.275104,2020-04-17
...,...,...,...
946,user_19,2020-05-15 10:22:39.698523,2020-05-15
947,user_19,2020-05-15 10:22:46.248162,2020-05-15
948,user_19,2020-05-15 10:23:18.043212,2020-05-15
949,user_28,2020-05-15 10:38:14.430013,2020-05-15


In [29]:
pivot_table = df.pivot_table(
  index='uid',
  columns='date',
  values='date',
  aggfunc='count',
  fill_value=0
)
pivot_table = pivot_table.cumsum(axis=1)
data = pivot_table.reset_index()
data

date,uid,2020-04-17,2020-04-18,2020-04-19,2020-04-22,2020-04-23,2020-04-24,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15
0,user_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,11
1,user_10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,21,58,58
2,user_11,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
3,user_12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4
4,user_13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,30,30,32,32
5,user_14,0,0,0,0,0,0,0,0,1,1,1,1,18,25,49,92,92,99,99
6,user_15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3
7,user_16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,9,9
8,user_17,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,5,5
9,user_18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7,8,31,31


In [30]:
numOfRows = data.shape[0]
numOfCols = data.shape[1] 
numOfFrames = numOfCols - 1
xaxis_range = [1,numOfFrames + 1]

initial_data = []
for uid_ind in range(numOfRows):
    y_axis = np.array(data.iloc[uid_ind,0])
    initial_data.append(go.Scatter(
        y = y_axis,
        mode = "lines+markers",
        name = data.uid[uid_ind]
    ))
initial_max = 0

# Frames
frames = []
for f in range(1,numOfFrames+1):
    curr_data = []
    title_names = []
    for uid_ind in range(numOfRows):
        curr_uid = data.uid[uid_ind]
        y_axis = np.array(data.iloc[uid_ind,1:f+1])
        curr_data.append(go.Scatter(
            x0=1,
            y = y_axis,
            mode = "lines+markers", 
            name = curr_uid,
            dy=10
        ))
    curr_frame = go.Frame(data = curr_data)
    frames.append(curr_frame)
 
figure = go.Figure(
    data = initial_data,
    frames = frames
)

figure.update_layout(
    title=dict(
        text="Dynamic of commits per user in project1",
        font=dict(size=30),
        x=0.5,
        y=0.95,
    ),
    xaxis=dict(
        showgrid=True, 
        dtick=2,
        range=xaxis_range
    ),
    yaxis=dict(
        showgrid=True, 
    ),
    legend=dict(
        x=1.1, 
        y=1,
        font=dict(size=12),
    ),
    width=1000,
    height=600,
    updatemenus=[
        {
            "type":"buttons",
            "buttons":[
                {
                    "method":"animate",
                    "label":"play",
                     "args":[None]
                }
            ]
        }
    ],
    paper_bgcolor='white',
    plot_bgcolor='rgba(220, 220, 220, 0.5)',  
)

figure.show()

In [31]:
con.close()